In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
import keras
import tensorflow as tf

Using TensorFlow backend.


In [3]:
seed_value = 0

np.random.seed(seed_value)
tf.set_random_seed(seed_value)

# 1 data

In [4]:
from keras.datasets import imdb

In [5]:
SQX_LEN = 100

In [6]:
NUM_WORD = 10000

In [7]:
imdb.load_data?

In [8]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(maxlen=SQX_LEN,num_words=NUM_WORD)

/Users/zhangjiaqi/workingspace/anaconda3/envs/ml3/lib/python3.7/site-packages/keras/datasets/imdb.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


In [9]:
word2id = imdb.get_word_index()

In [10]:
id2word = dict([(idx,word) for  word,idx in word2id.items()])

In [11]:
def get_word(sent):
    global id2word
    return ' '.join([id2word.get(i,'0') for i in sent])

In [12]:
print(' '.join([id2word[i] for i in x_train[0]]))

the effort still been that usually makes for of finished sucking ended and an because before if just though something know novel female i i slowly lot of above and with connect in of script their that out end his and i i


In [13]:
min(word2id.values())

1

In [14]:
##add start and end

In [15]:
word2id['\t'] = 0
id2word[0] = '\t'

In [16]:
from keras.preprocessing import sequence

In [17]:
sequence.pad_sequences?

In [18]:
x_train = sequence.pad_sequences(x_train,maxlen=SQX_LEN,padding='post')
x_test = sequence.pad_sequences(x_test,maxlen=SQX_LEN,padding='post')

In [19]:
x_train.shape

(5736, 100)

In [20]:
np.array((list(x_train[0][:-1])+[0])).shape

(100,)

In [21]:
def get_sample(x_train):
    x_train_vec = list()
    x_train_tar = list()
    for i in x_train:
        x_train_vec.append(list(i[:-1]) + [0])
        x_train_tar.append(list(i[1:])+[0])
    
    x_train_vec = np.array(x_train_vec)
    x_train_tar = np.array(x_train_tar)
    return x_train_vec,x_train_tar 

In [22]:
x_train_vec,x_train_tar = get_sample(x_train)
x_test_vec,x_test_var = get_sample(x_test)

In [23]:
print(get_word(x_train_tar[1]),x_train_tar[1])
print(get_word(x_train_vec[1]),x_train_vec[1])

no was personality as on was me time along exception not no was least that was station for of on with read is quite read been when was nature of before than do of hinted war with by br of sure of their with her perhaps that after another but is badly on type but of confusing not as by it his is must that it is producer return br stylish of white it much combination type but by 10 from after release perhaps on watch performances andre was own that breaking to is star some br already 	 	 [  54   13 1610   14   20   13   69   55  364 1398   21   54   13  219
   12   13 1706   15    4   20   16  329    6  176  329   74   51   13
  873    4  156   71   78    4 7412  322   16   31    7    4  249    4
   65   16   38  379   12  100  157   18    6  910   20  549   18    4
 1496   21   14   31    9   24    6  212   12    9    6 1322  991    7
 3002    4  425    9   73 2218  549   18   31  155   36  100  763  379
   20  103  351 5308   13  202   12 2241    5    6  320   46    7  457
    0    0]


In [24]:
x_train_tar = x_train_tar.reshape(x_train_tar.shape[0],x_train_tar.shape[1],1)

In [25]:
x_train_vec.shape

(5736, 100)

In [26]:
x_train_tar.shape

(5736, 100, 1)

# 2 model 

In [27]:
from keras.models import Model,Sequential
from keras.layers import Embedding, LSTM,Bidirectional,Dense,Input

In [28]:
input_x = Input(shape=(100,),name='input_x')

In [29]:
embed_x = Embedding(input_dim=NUM_WORD,output_dim=32)(input_x)

In [30]:
Bidirectional?

In [31]:
##lstm  batchsize* max_legth* 64
lstm1 = Bidirectional(LSTM(units=32,return_sequences=True))(embed_x)
lstm2 = Bidirectional(LSTM(units=32,return_sequences=True))(lstm1)

In [32]:
##out
out = Dense(units=NUM_WORD,activation=keras.activations.softmax)(lstm1)

In [33]:
nn = Model(input_x,out)

In [34]:
nn.compile(optimizer='adam',loss=keras.losses.sparse_categorical_crossentropy,metrics=[
    keras.metrics.categorical_accuracy,keras.metrics.categorical_crossentropy
])

In [35]:
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_x (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 32)           320000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 64)           16640     
_________________________________________________________________
dense_1 (Dense)              (None, 100, 10000)        650000    
Total params: 986,640
Trainable params: 986,640
Non-trainable params: 0
_________________________________________________________________


In [36]:
history = nn.fit(x_train_vec,x_train_tar,batch_size=256,epochs=10,validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 4588 samples, validate on 1148 samples
Epoch 1/10





4588/4588 [==============================] - 5016s 1s/step - loss: 9.1798 - categorical_accuracy: 0.8337 - categorical_crossentropy: 39998349.5937 - val_loss: 9.1096 - val_categorical_accuracy: 1.0000 - val_categorical_crossentropy: 39552016.8502
Epoch 2/10
4588/4588 [==============================] - 3833s 835ms/step - loss: 8.6327 - categorical_accuracy: 1.0000 - categorical_crossentropy: 40069465.3182 - val_loss: 7.6916 - val_categorical_accuracy: 1.0000 - val_categorical_crossentropy: 39961883.4704
Epoch 3/10
3584/4588 [======================>.......] - ETA: 41:55 - loss: 7.0482 - categorical_accuracy: 1.0000 - categorical_crossentropy: 41298435.7143 

KeyboardInterrupt: 

In [ ]:
x_train[0]

In [ ]:
np.array(x_train_tar).shape